In [1]:
import pandas as pd
import numpy as np

## 初探数据

In [2]:
uid_train = pd.read_csv('../data/uid_train.txt',sep='\t',header=None,names=('uid','label'))
voice_train = pd.read_csv('../data/voice_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_train = pd.read_csv('../data/sms_train.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_train = pd.read_csv('../data/wa_train.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [3]:
voice_test_a = pd.read_csv('../data/voice_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test_a = pd.read_csv('../data/sms_test_a.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test_a = pd.read_csv('../data/wa_test_a.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [4]:
voice_test_b = pd.read_csv('../data/voice_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','end_time','call_type','in_out'),dtype={'start_time':str,'end_time':str})
sms_test_b = pd.read_csv('../data/sms_test_b.txt',sep='\t',header=None,names=('uid','opp_num','opp_head','opp_len','start_time','in_out'),dtype={'start_time':str})
wa_test_b = pd.read_csv('../data/wa_test_b.txt',sep='\t',header=None,names=('uid','wa_name','visit_cnt','visit_dura','up_flow','down_flow','wa_type','date'),dtype={'date':str})

In [5]:
def getVoiceDura(startTime, endTime):
    #start time
    str1 = str(startTime).zfill(8)
    day1 = int(str1[0:2])
    hour1 = int(str1[2:4])
    minute1 = int(str1[4:6])
    second1 = int(str1[6:8])
    #end time
    str2 = str(endTime).zfill(8)
    day2 = int(str2[0:2])
    hour2 =  int(str2[2:4])
    minute2 = int(str2[4:6])
    second2 = int(str2[6:8])
    if (day2 > day1):
        hour2 += (day2 - day1) * 24
    duraTime = (hour2  - hour1) * 3600 + (minute2 - minute1) * 60 + (second2 - second1)
    return duraTime

In [6]:
def getMaxOppNum(ylist):
    for tmp in ylist:
        maxtimes = 0
        times = 0
        for dlist in ylist:
            if tmp == dlist:
                times += 1
        if maxtimes < times:
            maxtimes = times
    return maxtimes

In [7]:
print(voice_train.info())
print(sms_train.info())
print(wa_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1150778 entries, 0 to 1150777
Data columns (total 8 columns):
uid           1150778 non-null object
opp_num       1150778 non-null object
opp_head      1150778 non-null int64
opp_len       1150778 non-null int64
start_time    1150778 non-null object
end_time      1150778 non-null object
call_type     1150778 non-null int64
in_out        1150778 non-null int64
dtypes: int64(4), object(4)
memory usage: 70.2+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 302976 entries, 0 to 302975
Data columns (total 6 columns):
uid           302976 non-null object
opp_num       302976 non-null object
opp_head      302976 non-null int64
opp_len       302976 non-null int64
start_time    302976 non-null object
in_out        302976 non-null int64
dtypes: int64(3), object(3)
memory usage: 13.9+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4808343 entries, 0 to 4808342
Data columns (total 8 columns):
uid           object
wa_name       

## 获得测试集A的uid

In [8]:
uid_test_a = pd.DataFrame({'uid':pd.unique(wa_test_a['uid'])})
uid_test_a.to_csv('../data/uid_test_a.txt',index=None)

## 获得测试集B的uid

In [9]:
uid_test_b = pd.DataFrame({'uid':pd.unique(wa_test_b['uid'])})
uid_test_b.to_csv('../data/uid_test_b.txt',index=None)

## 合并训练集和测试集

In [10]:
voice = pd.concat([voice_train,voice_test_a,voice_test_b],axis=0)
sms = pd.concat([sms_train,sms_test_a,sms_test_b],axis=0)
wa = pd.concat([wa_train,wa_test_a,wa_test_b],axis=0)

In [11]:
#qqq = voice_time_dura = voice.groupby(['uid','opp_len'])['uid', 'voice_time_last']
#print(qqq.head(10))

##  baseline

### 通话记录

In [12]:
# 每个用户的通话号码数
voice_opp_num = voice.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count','max_times': lambda x: getMaxOppNum(x)}).add_prefix('voice_opp_num_').reset_index()

voice_opp_head = voice.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('voice_opp_head_').reset_index()

voice['opp_head'] = voice['opp_head'].astype('float64')

voice_opp_heads = voice.groupby(['uid'])['opp_head'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_opp_head_').reset_index()

voice_opp_len = voice.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('voice_opp_len_').reset_index().fillna(0)

voice['opp_len'] = voice['opp_len'].astype('float64')

voice_opp_lens = voice.groupby(['uid'])['opp_len'].agg(['std','max','min','median','mean','sum']).add_prefix('voice_opp_len_').reset_index()

voice['voice_time_last'] = voice.apply(lambda t: getVoiceDura(t['start_time'], t['end_time']), axis=1)

voice_time_dura = pd.DataFrame({'uid': voice['uid'], 'voice_time_dura':voice['voice_time_last']})

voice_call_type = voice.groupby(['uid','call_type'])['uid'].count().unstack().add_prefix('voice_call_type_').reset_index().fillna(0)

voice_in_out = voice.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('voice_in_out_').reset_index().fillna(0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  after removing the cwd from sys.path.


### 短信记录

In [13]:
sms_opp_num = sms.groupby(['uid'])['opp_num'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count','max_times': lambda x: getMaxOppNum(x)}).add_prefix('sms_opp_num_').reset_index()

sms_opp_head = sms.groupby(['uid'])['opp_head'].agg({'unique_count': lambda x: len(pd.unique(x))}).add_prefix('sms_opp_head_').reset_index()

sms['opp_head'] = sms['opp_head'].astype('float64')

sms_opp_heads = sms.groupby(['uid'])['opp_head'].agg(['std','max','min','median','mean','sum']).add_prefix('sms_opp_head_').reset_index()

sms_opp_len = sms.groupby(['uid','opp_len'])['uid'].count().unstack().add_prefix('sms_opp_len_').reset_index().fillna(0)

sms['opp_len'] = sms['opp_len'].astype('float64')

sms_opp_lens = sms.groupby(['uid'])['opp_len'].agg(['std','max','min','median','mean','sum']).add_prefix('sms_opp_len_').reset_index()

sms_in_out = sms.groupby(['uid','in_out'])['uid'].count().unstack().add_prefix('sms_in_out_').reset_index().fillna(0)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until


### 网站/APP记录

In [14]:
wa_name = wa.groupby(['uid'])['wa_name'].agg({'unique_count': lambda x: len(pd.unique(x)),'count':'count','max_times': lambda x: getMaxOppNum(x)}).add_prefix('wa_name_').reset_index()

visit_cnt = wa.groupby(['uid'])['visit_cnt'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_cnt_').reset_index()

visit_dura = wa.groupby(['uid'])['visit_dura'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_visit_dura_').reset_index()

up_flow = wa.groupby(['uid'])['up_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_up_flow_').reset_index()

down_flow = wa.groupby(['uid'])['down_flow'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_down_flow_').reset_index()

wa_type = wa.groupby(['uid','wa_type'])['uid'].count().unstack().add_prefix('wa_type_').reset_index().fillna(0)

wa['date'] = wa['date'].astype(float)

wa_date = wa.groupby(['uid'])['date'].agg(['std','max','min','median','mean','sum']).add_prefix('wa_date_').reset_index()

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.


## 提取特征

In [15]:
feature = [voice_opp_num,voice_opp_head,voice_opp_heads,voice_opp_len,voice_opp_lens,voice_time_dura,voice_call_type,voice_in_out,
           sms_opp_num,sms_opp_head,sms_opp_heads,sms_opp_len,sms_opp_lens,sms_in_out,
           wa_name,visit_cnt,visit_dura,up_flow,down_flow,wa_type,wa_date]

## 拆分训练集测试集特征

In [16]:
train_feature = uid_train
for feat in feature:
    train_feature = pd.merge(train_feature,feat,how='left',on='uid')

In [17]:
test_feature_a = uid_test_a
for feat in feature:
    test_feature_a = pd.merge(test_feature_a,feat,how='left',on='uid')

In [18]:
test_feature_b = uid_test_b
for feat in feature:
    test_feature_b = pd.merge(test_feature_b,feat,how='left',on='uid')

## 将特征保存为csv文件形式

In [19]:
train_feature.to_csv('../data/train_feature_V1.csv',index=None)
test_feature_a.to_csv('../data/test_feature_a_V1.csv',index=None)
test_feature_b.to_csv('../data/test_feature_b_V1.csv',index=None)